In [14]:
# Note: right now, this script is stored under the post_macros 
# folder. look at comments in 'inputs' section below for 
# places to change relative filepaths

#################################
## -- INPUTS/IMPORT MODULES -- ##
#################################

#libraries
import os
import numpy as np, pandas as pd


#output locations
workspace = os.path.dirname(os.getcwd())            # <<-- 'Database' folder
output_location = workspace + '\\data'              # <<-- 'data' folder, where punchmoves has been written previously
punchlink_out = output_location + '\\punchlink.csv' # <<-- output csv name


#import emme desktop and initialize emme
import inro.emme.desktop.app as _app
cwd = os.getcwd()
e = os.listdir(os.path.dirname(workspace))
empfile = [os.path.join(os.path.dirname(workspace), file) for file in e if file.endswith('.emp')][0] # <<-- 'emp' file of model run


desktop = _app.start_dedicated(
    visible=False,
    user_initials="cmap",
    project=empfile
)

#import modeller
import inro.modeller as _m
modeller = _m.Modeller(desktop=desktop)
emmebank = modeller.emmebank

In [ ]:

#emme tool
net_calc = modeller.tool('inro.emme.network_calculation.network_calculator')
create_attribute = modeller.tool('inro.emme.data.extra_attribute.create_extra_attribute')
copy_scenario = modeller.tool('inro.emme.data.scenario.copy_scenario')
delete_scenario = modeller.tool('inro.emme.data.scenario.delete_scenario')

# -- 



In [ ]:

###############################
## -- EXECUTE PUNCH MOVES -- ##
###############################

print('EXTRACT ROADWAY LINK ATTRIBUTES')

## -- SPECS FOR FLAGGING TOLL LINKS

#zero out ui1
ui1spec = {
    "result": "ui1",
    "expression": "0",
    "selections": {"node":"all"},
    "type": "NETWORK_CALCULATION"
}

#zero out ui2
ui2spec = {
    "result":"ui2",
    "expression":"0",
    "selections": {"node":"all"},
    "type": "NETWORK_CALCULATION"
}

#flag ramps (vdf 3,5,8) in ui1
ui1spec2 = {
    "result": "ui1",
    "expression": "vdf==3 .or. vdf==5 .or. vdf==8",
    "aggregation": ".max.",
    "selections": {"link": "all"},
    "type": "NETWORK_CALCULATION"
}

#flag ramps (vdf 3,5,8) in ui2
uj2spec2 = {
    "result": "uj2",
    "expression": "vdf==3 .or. vdf==5 .or. vdf==8",
    "aggregation": ".max.",
    "selections": {"link": "all"},
    "type": "NETWORK_CALCULATION"
}


#flag toll links (vdf=7) on ramps (vdf 3,5,8)
#based on whether a ramp exists on either side of the link
tmpl2spec = '''
{
    "result": "@tmpl2",
    "expression": "(ui2+uj1).ge.2",
    "selections": {"link": "vdf=7"},
    "type": "NETWORK_CALCULATION"
}
'''


#list of attributes to extract for punchlink
desired_links = '''\
    "length+lanes+vdf+\
    @zone+@emcap+timau+\
    @ftime+@avauv+@avh2v+\
    @avh3v+@avbqv+@avlqv+\
    @avmqv+@avhqv+@busveq+\
    @atype+@imarea+@tmpl2\
    @speed+@m200+@h200"\
'''

In [ ]:
#container to hold dataframe for each timeperiod
df_list = []


#iterate through each time period (1-8)
for tp in range(1,9):
    print(f'  -- Obtaining link data for time period {tp}...')

    copy_scenario(
        from_scenario=tp,
        scenario_id=99998,
        scenario_title='temp copy for MOVES',
        overwrite=True
    )


    #flag toll links on ramps
    net_calc(specification=[ui1spec, ui2spec], scenario=emmebank.scenario(99998))
    net_calc(specification=[ui1spec2,uj2spec2], scenario=emmebank.scenario(99998))
    
    #create attribute @tmpl2 to hold toll flag
    create_attribute(
        extra_attribute_type='LINK',
        extra_attribute_name='@tmpl2',
        extra_attribute_description='toll link on ramp',
        extra_attribute_default_value=0,
        overwrite=True,
        scenario=emmebank.scenario(99998)
    )
    net_calc(specification=tmpl2spec, scenario=emmebank.scenario(99998))

    #list of attributes to extract for punchlink
    desired_links = '''"length+lanes+vdf+@zone+@emcap+timau+@ftime+@avauv+@avh2v+@avh3v+@avbqv+@avlqv+@avmqv+@avhqv+@busveq+@atype+@imarea+@tmpl2+@speed+@m200+@h200"'''
    
    spec_linkdata = f'''
    {{
        "expression": {desired_links},
        "selections": {{"link":"all"}},
        "type": "NETWORK_CALCULATION"
    }}
    '''
    #network calculation to export attributes
    linkdata_tp = net_calc(
        specification=spec_linkdata, 
        scenario=emmebank.scenario(99998), 
        full_report=True
    )

    header = linkdata_tp['table'][0]
    data = linkdata_tp['table'][1:]

    linkdata_tp_df = pd.DataFrame(data=data, columns=header)
    linkdata_tp_df['timeperiod'] = tp
    
    df_list.append(linkdata_tp_df)

linkdata = pd.concat(df_list, ignore_index=True)

columns=linkdata.columns.tolist()
rename={}
for c in columns:
    if c.startswith('@'):
        rename[c] = c[1:]
rename['tmpl1'] = 'timeperiod'
linkdata.rename(columns=rename, inplace=True)

linkdata.to_csv(punchlink_out, index=False)
print(f'Done! Output csv located here: {punchlink_out}')

In [ ]:
desktop.close()

In [ ]:
## -- QUALITY CONTROL -- ##
###########################

og_punch_files = [] 
for x in range(1,9):
    punch_file = output_location + f'\\moves_pd{x}.data' 
    punch = pd.read_csv(punch_file, delim_whitespace=True, encoding='utf-8')
    og_punch_files.append(punch)
og_punch = pd.concat(og_punch_files, ignore_index=True)
og_punch.rename(
    columns={
        'inode':'i_node',
        'jnode':'j_node',
        'tmpl1':'timeperiod'
    },
    inplace=True
)

rename = {}
columns = og_punch.columns.tolist()
for c in columns:
    if c.startswith('@'):
        rename[c] = c[1:]
og_punch.rename(columns=rename, inplace=True)

print('-- DATA CHECKS -- \n')

#number of rows
print('NUMBER OF ROWS:\n')
print('Number of rows in new punchlink: ' + str(len(linkdata)) + '\nNumber of rows in original punchlink: ' + str(len(og_punch)))

if str(len(linkdata)) == str(len(og_punch)):
    print('Same number of rows for old punchmoves and new punchmoves.\n')
else:
    print('Different number of rows for old punchmoves and new punchmoves. Look for scripting mishap!\n')

print('LINK-TIMEPERIOD PAIRS:\n')
join = pd.merge(linkdata, og_punch, how='inner', on=['i_node', 'j_node', 'timeperiod'])
print('Number of rows on an inner join of inode, jnode, and timeperiod: ' + str(len(join)))

if str(len(join)) == str(len(linkdata)) and str(len(join)) == str(len(og_punch)):
    print('Each timeperiod-link pair exists in both old and new: the new script grabbed all appropriate links correctly!\n')
else:
    print('Missing timeperiod-link pairs in new punchlink! Review how the new script grabben links.\n')

## -- 

#column totals
print('DIFFERENCES IN TOTAL VOLUME: \n')

df = pd.DataFrame()
newsum = linkdata.drop(columns=['m200','h200','result']).sum()
df['new'] = newsum

oldsum = og_punch.drop(columns=['result']).sum()
df['old'] = oldsum

df.eval('difference = new - old', inplace=True)
df['significance'] = np.where(abs(df['difference'])>0.0001, 1, 0)

rows = df['significance'].sum()
print(f'There are {rows} categories that have more than 0.0001 difference in regional summed values (for volume or any other measure).\n')
if rows!=0:
    print(f'Closer look may be necessary.')

df
